**Megan Rochella - 0706022210028**

In [54]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [55]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')

df.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [56]:
# flatten the dataframe and extract unique items (products)
unique_products = set(df.values.flatten())

# show unique products
print(unique_products)

{'Eggs', nan, 'Milk', 'Bread', 'Bagel', 'Meat', 'Cheese', 'Wine', 'Diaper', 'Pencil'}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [57]:
# create an itemset based on the products
products = set()
for col in df.columns:
    products.update(df[col].unique())

# encoding the feature
encoded_transactions = []
for _, row in df.iterrows():
    transaction_dict = {product: (1 if product in row.values else 0) for product in products}
    encoded_transactions.append(transaction_dict)

# show encoded transaction
print("Encoded Transaction Representation")
encoded_transactions[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Encoded Transaction Representation


{'Eggs': 1,
 nan: 0,
 'Milk': 0,
 'Bread': 1,
 'Bagel': 0,
 'Meat': 1,
 'Cheese': 1,
 'Wine': 1,
 'Diaper': 1,
 'Pencil': 1}

In [58]:
# create new dataframe from the encoded features
# replace NaN with a specific value, e.g., 'No Product'
df_with_nan = df.fillna('NaN')

# flatten the data again, but keep 'No Product' as a category
flattened_data = df_with_nan.values.flatten()

# use OneHotEncoder, including 'No Product' which represents missing values
# replace 'sparse' with 'sparse_output'
# make sure unknown values are ignored
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(flattened_data.reshape(-1, 1))

# create the one-hot encoded DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.categories_[0])

# create a DataFrame with 0s and 1s (representing whether the item is bought)
product_data = pd.DataFrame(0, index=df.index, columns=encoder.categories_[0])

# loop through each row and mark presence of products, including 'No Product'
for i, row in df_with_nan.iterrows():
    for product in row:
        product_data.loc[i, product] = 1

# show encoded dataframe
print("Encoded DataFrame:")
product_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Encoded DataFrame:


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,NaN,Pencil,Wine
0,0,1,1,1,1,1,0,0,1,1
1,0,1,1,1,0,1,1,0,1,1
2,0,0,1,0,1,1,1,1,0,1
3,0,0,1,0,1,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1,1


In [59]:
# since, the encoded dataframe consist of the empty column, we will drop the NaN column or u can use the index
# removing if 'NaN' column appears
if 'NaN' in product_data.columns:
    product_data.drop(columns=['NaN'], inplace=True)

# show cleaned DataFrame
print("Encoded DataFrame (with NaN column dropped):")
product_data.head()

Encoded DataFrame (with NaN column dropped):


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [60]:
# set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules

# apply the apriori algorithm to find frequent itemsets
min_support = 0.2
frequent_itemsets = apriori(product_data, min_support=min_support, use_colnames=True)

# show frequent itemsets
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bread, Bagel)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [61]:
# set the minimum threshold for confidence
confidence_threshold = 0.6

# generate association rules based on the frequent itemsets, using "confidence" as the metric
# the `metric="confidence"` specifies that the rules will be filtered based on their confidence values
# the `min_threshold=confidence_threshold` ensures that only rules with confidence >= 0.6 are included
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_threshold)

# drop the column 'zhangs_metric' from the rules DataFrame
rules.drop(columns=['zhangs_metric'], inplace=True)

# show resulting association rules
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
2,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
3,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
4,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
5,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Eggs, Meat)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667
9,"(Eggs, Cheese)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

# **Explanation**
1. Support:
- Support is a measure of how frequently a particular itemset (or combination of items) appears in the dataset.
- It represents the proportion of transactions in the dataset that contain a particular combination of items.
- Support tells you how common or rare a particular itemset is in the dataset. High support means the itemset is frequently seen, while low support means it is rare.

2. Antecedent Support and Consequent Support:
- The antecedent (or left-hand side) refers to the items that appear on the left side of the rule, and the consequent (or right-hand side) refers to the items on the right side of the rule.
- The support for the antecedent refers to how often the items in the antecedent appear together in transactions.
- The support for the consequent refers to how often the items in the consequent appear together in transactions.

3. Confidence:
- Confidence is a measure of the likelihood that the consequent occurs when the antecedent occurs. In other words, it represents the conditional probability of the consequent given the antecedent.
- A high confidence means that when item A is purchased, item B is very likely to be purchased as well. A confidence of 1 means the rule is always true.

4. Lift:
- Lift measures how much more likely the consequent is to occur when the antecedent occurs, compared to when the antecedent and consequent are independent. It is a way of evaluating the strength of a rule by comparing it to a baseline (the expected frequency of the consequent in the dataset).

5. Leverage:
- Leverage quantifies the difference between the observed frequency of the itemset occurring together and the expected frequency if the items were independent. It tells you how much more or less likely it is that the items appear together than would be expected by chance.

6. Conviction:
- Conviction is a measure of how strongly the presence of the antecedent implies the presence of the consequent.
- It takes into account the likelihood that the rule would occur if the antecedent did not lead to the consequent.

References:

- Rahmadsyah, A., & Mayasari, N. (2022). Grouping Goods with the Association Rule Method Using A priori Algorithms in Modern Retail Stores. International Journal of Science, Technology & Management, 3(6), 1527-1532. https://doi.org/10.46729/ijstm.v3i6.637

- Hossain, M. A., & Choi, M. (2021). Mining Association Rules with Application to Predictive Modeling: A Review. IEEE Access, 9, 109343-109368.

- Sharma, S., & Gupta, M. (2021). Optimization of Association Rule Mining Algorithms Using Lift and Leverage Metrics for E-Commerce Applications. Journal of Retailing and Consumer Services, 60, 102443.

- Pandey, M., & Sharma, M. (2023). A Hybrid Approach for Association Rule Mining in Data Streams Using Confidence and Conviction Metrics. Computers, Materials & Continua, 74(3), 3387-3404.

- GPT LINK https://chatgpt.com/share/6748372e-d39c-800d-a7a6-85934792ac5b

# **Interpretation**
1. Strong Rules:
- (Eggs, Meat) → (Cheese) has high confidence (0.809524) and a lift (1.613924), meaning that customers who buy Eggs and Meat together are highly likely to buy Cheese.
- (Milk, Cheese) → (Meat) has a lift (1.400000) and a confidence (66.67%), indicating a strong association between these items.

2. Weaker Rules:
- (Bagel) → (Bread) has a lift (1.301042) and relatively low confidence (65.67%), showing a weaker correlation compared to other rules.
- (Wine) → (Cheese) has a lift (1.227986) and low confidence (61.59%), suggesting a weaker relationship.

3. High Conviction:
- (Meat, Milk) → (Cheese) shows a conviction of 2.952137, suggesting that this rule is particularly reliable for predicting Cheese purchases when Meat and Milk are bought.

4. Combination Rules:
- Rules with multiple antecedents, such as (Eggs, Meat) → (Cheese), often have stronger metrics (higher confidence and lift) compared to single antecedent rules.
- This highlights the importance of combinations in predicting customer behavior.

5. Personal Reccomendations:
- Cheese appears to be a frequent co-occurring item, often purchased with products like Milk, Eggs, and Meat. Promotions bundling these items could boost sales.
- Pairing Eggs and Meat with a recommendation for Cheese may increase cross-selling opportunities.
- Products like Wine and Cheese show some relationship, though weaker. Targeted promotions for these could appeal to niche customer segments.

6. My Conclusion:
- This analysis highlights potential cross-selling opportunities, where understanding which products are often bought together can help inform marketing strategies, store layouts, and promotional offers. The metrics suggest that products like Cheese, Meat, and Eggs and Milk, Cheese, and Meat tend to appear together in transactions, making them strong candidates for bundling or targeted marketing campaigns.